# 数据载入

In [42]:
import pandas as pd
import os
from collections import Counter
import numpy as np

import torch

In [43]:
pd.set_option('display.max_colwidth', -1)

C:\Users\c'y'y\AppData\Local\Temp\ipykernel_18852\3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [44]:
poi_path =r'E:\xpj\research\POI\psqlcsv\beijing.csv'
poi_prompt_path =r'E:\xpj\research\POI\psqlcsv\beijing_poi_prompt.csv'
image_path = r'E:\xpj\research\POI\psqlcsv\beijing_img_location.csv'
image_poi_path = r'E:\xpj\research\POI\psqlcsv\beijing_img_poi2.csv'
desc_path =r'E:\xpj\research\POI\desc_merge\Beijing_img_desc_clean.csv'
best_poi_path = r'E:\xpj\research\POI\result\beijing_best_poi.csv'
desc_poi_path = r'E:\xpj\research\POI\result\beijing_desc_poi.csv'

## poi数据（全面）

In [45]:
poi_data = pd.read_csv(poi_path,encoding='utf-8')
# poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_poi.csv",encoding='ANSI')
poi_df = pd.DataFrame(poi_data)

d:\software\Anaconda\envs\clip\lib\site-packages\IPython\core\interactiveshell.py:3269: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [46]:
poi_df

,id,geom,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags
0,1,0101000020E61000000C4169030B195D406EC5596CEEF34340,25248662,北京市,NaN,NaN,京,NaN,NaN,city,NaN,"""admin_level""=>""2"",""alt_name""=>""北京"",""alt_name:cs""=>""Pej-ťing"",""alt_name:en""=>""Peking;Peiping Municipality;Pei-p'ing Shih;Peking Municipality;Peiping Municipal Administrative Area"",""alt_name:eo""=>""Bejĝino"",""alt_name:fr""=>""Beijing"",""alt_name:is""=>""Pe"
1,2,0101000020E6100000EB1F4432E4185D40F85BF1C3F7F34340,25248785,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
2,3,0101000020E6100000B43808DF20195D40A93C7084FBF34340,25248786,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
3,4,0101000020E6100000AAB8718BF9185D403C69E1B20AF34340,25248790,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
4,5,0101000020E6100000C851CAB532195D40D44911740FF34340,25248791,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
...,...,...,...,...,...,...,...,...,...,...,...,...
96102,96103,0101000020E6100000685E6974621D5D408382F7FAFFFF4340,11382821942,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
96103,96104,0101000020E6100000EC51B81E851D5D407F64DFBA07004440,11382821943,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
96104,96105,0101000020E610000030BEC3488A1D5D40C761D5C512004440,11382821944,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
96105,96106,0101000020E6100000C9FDB38B8E1D5D400AB6B69503004440,11382821945,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals""=>""signal"""


In [47]:
poi_df.columns

Index(['id', 'geom', 'osm_id', 'name', 'barrier', 'highway', 'ref', 'address',
       'is_in', 'place', 'man_made', 'other_tags'],
      dtype='object')

 # 下面第一块可以不做

In [7]:
# save tag list to txt

# poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway','ref', 'address', 'place', 'man_made', 'other_tags']]
# poi_df_ot =poi_df_nec.loc[:,['other_tags']]
# tag_list = []
# def collect_tags(x):
#     # print(x['other_tags'])
    
#     if  pd.notna(x['other_tags']):
#         # print(x['other_tags'])
#         tags = x['other_tags'].split("\",\"")
#         # print(tags)
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 if t not in tag_list:
#                     tag_list.append(t)
#             except:
#                 pass
#     # return tag_list

# result = poi_df_ot.apply(lambda x: collect_tags(x), axis=1, result_type='expand')
# # result
# tag_list.sort()
# tag_list
# # 保存tag_list到txt
# with open(r'E:\xpj\research\POI\chennai_tag_list.txt', 'w') as f:
#     for item in tag_list:
#         f.write("%s\n" % item)

# poi_df_ot

# def genSent(x):
#     tags = None
#     b = True    
#     # sen = "The imformation of this point contains that "
#     sen = "Nearby,"
#     for key in ['name','barrier', 'highway','address','place','man_made']:
#         if not x[key].isna().iloc[0]:
#             # print(x[key])
#             sen = sen + key +" is " + x[key].iloc[0] + ","
#     if not x['other_tags'].isna().iloc[0]:
#         tags = x['other_tags'].iloc[0].split("\",\"")
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 k = tag.split("=>")[1]
#                 if "name:" in t and t != "name:en":
#                     pass
#                 sen = sen + t +" is " + k + ","
#             except:
#                 pass

#     sen = sen + "located in " + x['NAME_2'].iloc[0]
    
#     # return sen,b
    
#     return sen
# # poi_df_nec


## 照片信息

In [48]:
pic_data = pd.read_csv(image_path,encoding='utf-8')
# pic_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_points.csv",encoding='ANSI')
pic_df = pd.DataFrame(pic_data)

In [49]:
pic_df

,id,geom,captured_a,camera_mak,created_at,sequences_,pano,user_key,username,img_key,ca,longitude,latitude
0,93944,0101000020E61000001BDA006C40115D400B07E7F80D004440,2017/09/29 05:21:12.635,motorola,2017/10/05 17:23:23.266,arlEzB8exOI9q2Ny_rwCEw,0,la-f4O0CvbqsbdbrOaVraQ,mapeadora,okgNJebzh56b4p0HGLPW8A,33.3929,116.269557,40.000426
1,93945,0101000020E610000009B42A7840115D406A6B44300E004440,2017/09/29 05:21:12.635,motorola,2017/10/05 17:23:23.266,arlEzB8exOI9q2Ny_rwCEw,0,la-f4O0CvbqsbdbrOaVraQ,mapeadora,8rY_L5Hlrl5eUM8LaJ6F-g,344.3040,116.269560,40.000433
2,93946,0101000020E610000074DD4A6540115D404CAEBDAA0E004440,2017/09/29 05:21:12.635,motorola,2017/10/05 17:23:23.266,arlEzB8exOI9q2Ny_rwCEw,0,la-f4O0CvbqsbdbrOaVraQ,mapeadora,J3Bxw4h6SkRbrIvIfkENbg,343.7920,116.269555,40.000448
3,93947,0101000020E610000074C6415340115D40993160240F004440,2017/09/29 05:21:12.635,motorola,2017/10/05 17:23:23.266,arlEzB8exOI9q2Ny_rwCEw,0,la-f4O0CvbqsbdbrOaVraQ,mapeadora,HDeYpM-t0r7k8MOleHCc2Q,58.5378,116.269551,40.000462
4,93948,0101000020E6100000DF1D746440115D405D75D31C0F004440,2017/09/29 05:21:12.635,motorola,2017/10/05 17:23:23.266,arlEzB8exOI9q2Ny_rwCEw,0,la-f4O0CvbqsbdbrOaVraQ,mapeadora,Ec2AEq5yN8b3f2ivdbrsxQ,115.3060,116.269555,40.000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157997,93939,0101000020E61000003FD7080E79185D404427953A12F84340,2017/09/30 13:23:02.000,Garmin,2018/01/20 02:23:41.385,1asso0i9d44o52qnn5jbhq,0,la-f4O0CvbqsbdbrOaVraQ,mapeadora,oM2JyOQmXs-puWEAd4XWiQ,29.2621,116.382389,39.938056
157998,93940,0101000020E61000000D33349E881E5D40ADE584BF15EC4340,2017/03/15 02:33:21.011,OPPO,2017/03/15 11:47:31.729,XznMuNIcaeSzyNu6M--3cg,0,Ju0Sxwe4llSvnze1Hi0ijA,cdch414,Eo2jDXQbPk9DbeELipkFEQ,37.5983,116.477088,39.844414
157999,93941,0101000020E61000007859B851891E5D40E86DC72F17EC4340,2017/03/15 02:33:21.011,OPPO,2017/03/15 11:47:31.729,XznMuNIcaeSzyNu6M--3cg,0,Ju0Sxwe4llSvnze1Hi0ijA,cdch414,VjNZtHvD1hN9H5vCY9W3TQ,38.0567,116.477131,39.844458
158000,93942,0101000020E610000091D61874C23E5D4020425C397BE34340,2017/04/09 14:02:51.000,OPPO,2017/04/09 19:58:46.456,PP2_02z4R9NaCb6R6G2NbA,0,Ju0Sxwe4llSvnze1Hi0ijA,cdch414,5M4anLCfa04Qj3b5bQS6Yw,336.1080,116.980619,39.777198


## 照片poi关联

In [50]:
pic_poi_data = pd.read_csv(image_poi_path,encoding='utf-8')
# pic_poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_pois_to_points.csv",encoding='ANSI')
pic_poi_df = pd.DataFrame(pic_poi_data)

In [51]:
pic_poi_df

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,distance5
0,002X1xuI4XOHBir7-owySg,11362,79.865680,70509.0,82.517518,70507.0,90.497904,70508.0,96.740065,59698.0,113.766971
1,003EDEceaZbom1pL5BT71g,36664,70.387451,75413.0,235.091576,36665.0,324.879509,21346.0,376.064179,21348.0,378.348055
2,005mgSHE0DfmONwbkZrE5A,39296,79.438995,39354.0,116.160086,39295.0,117.426100,38720.0,161.605241,38719.0,163.052673
3,006xgYXlcHgZ8BfTttGuIA,58185,66.851118,11517.0,80.813306,58178.0,83.479790,54082.0,84.786197,11519.0,87.400682
4,007x7NqVxRpQWenELoWNiw,27457,81.035875,49286.0,155.152377,27564.0,166.277333,44452.0,167.483308,27498.0,180.348641
...,...,...,...,...,...,...,...,...,...,...,...
132873,ZzZb1yoT-DYIk-Bvb1KI_Q,82356,24.981663,7072.0,37.628208,7073.0,39.040349,2692.0,39.958836,82355.0,45.165065
132874,Zzzl07EjELpMNuScHuWvOQ,85607,309.203806,85608.0,318.729493,85610.0,332.109423,85609.0,339.779948,NaN,NaN
132875,ZZzwGOYsu3J8NwRqR7dvGg,20801,7.442443,21698.0,30.921744,19616.0,68.250814,15765.0,121.839202,22496.0,146.615834
132876,ZzZX6Ac03dml7i9NUiabGg,18739,5.880840,18734.0,8.979344,18707.0,16.072906,18717.0,17.658410,18713.0,23.614230


## 照片描述

In [52]:
desc_pd =pd.read_csv(desc_path,encoding='utf-8')
desc_pd

,key,prompt
0,10pghDAJsxu1sdFN-xwKNg,"there is a cat sitting on the steps in front of a building, a detailed matte painting inspired by Shin Saimdang, tumblr, cloisonnism, korean traditional palace, beautiful ancient ruins behind, in forbidden city"
1,gvh8c4gE4__oDvbB46QhYw,"araffe with a lot of people walking around it, a photo inspired by Gong Xian, tumblr, cloisonnism, ancien chinese tower, taoist temples and monks, chinese architecture"
2,gGhf1zLc7LNTG_xNCU-P4w,"arafed roof of a building with a view of a river and trees, a detailed matte painting inspired by Gong Xian, tumblr, cloisonnism, imperial city in the distance, chinese palace, ancien chinese tower"
3,kYJWkBuWPPrjkT7bhBcIvQ,"there is a view of a building with a green roof and a yellow railing, a matte painting by Emperor Huizong of Song, cg society, cloisonnism, heise jinyao, taoist temples and monks, pagodas on hills"
4,zEp8lLNHY4m4kNjdI0Liow,"people walking up a steep staircase with a view of a lake, a matte painting inspired by Chen Daofu, dribbble, cloisonnism, 400 steps, taoist temples and monks, pagodas on hills"
...,...,...
157997,ClJqUg9jNDQh9ORQ6ufn7A,"cars are driving down a snowy street in a city, a matte painting by Muqi, polycount, dau-al-set, baotou china, artificial fog, sky is not visible"
157998,XbHTwE8c6KY-NXyAc-Ur2Q,"snowy street with cars driving on it in the middle of the day, a matte painting by Weiwei, tumblr, constructivism, neo norilsk, baotou china, soviet suburbs"
157999,WfvUbbt2GKVF29yV0rcupg,"arafed view of a highway with a foggy sky and street lights, a matte painting by Ye Xin, tumblr, tonalism, foggy photo 8 k, gloomy and foggy atmosphere, gloomy foggy atmosphere"
158000,xIBlWSCw8BVvPzlv8q1cJw,"arafed view of a car driving down a snowy road, a matte painting inspired by Zhang Kechun, tumblr, tonalism, in winter, heise jinyao, winter photograph"


# 数据处理

## poi数据预处理

In [53]:
poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway', 'address', 'place', 'man_made', 'other_tags']]

In [54]:
poi_df_nec

,id,name,barrier,highway,address,place,man_made,other_tags
0,1,北京市,NaN,NaN,NaN,city,NaN,"""admin_level""=>""2"",""alt_name""=>""北京"",""alt_name:cs""=>""Pej-ťing"",""alt_name:en""=>""Peking;Peiping Municipality;Pei-p'ing Shih;Peking Municipality;Peiping Municipal Administrative Area"",""alt_name:eo""=>""Bejĝino"",""alt_name:fr""=>""Beijing"",""alt_name:is""=>""Pe"
1,2,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
3,4,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
4,5,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward"""
...,...,...,...,...,...,...,...,...
96102,96103,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
96103,96104,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
96104,96105,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
96105,96106,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal"""


In [ ]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec.csv")

# 硬提示计算相似度，找最佳poi

In [55]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# 加载BERT模型和分词器
tokenizer = AutoTokenizer.from_pretrained(r"E:\xpj\models\bert-base-uncased")
model = AutoModel.from_pretrained(r"E:\xpj\models\bert-base-uncased")

In [56]:
# 定义计算相似度的函数
def calc_similarity(s1, s2):
    # 对句子进行分词，并添加特殊标记
    inputs = tokenizer([s1, s2], return_tensors='pt', padding=True, truncation=True)
    # print(inputs)

    # 将输入传递给BERT模型，并获取输出
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    # 计算余弦相似度，并返回结果
    sim = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return sim


In [81]:

def isEnglish(s):
    if isinstance(s, str):
        return s.isascii()
    return False



# 从txt中读取tag_list
tag_ignore_list = []
with open(r'E:\xpj\research\POI\tag_ignore_list.txt', 'r') as f:
    for line in f.readlines():
        tag_ignore_list.append(line.strip())
# print(tag_ignore_list)

def genSent(x):
    tags = None
    b = True    
    # sen = "The imformation of this point contains that "
    sen = "Nearby,"
    for key in ['name','barrier', 'highway','address','place','man_made']:
        # if isinstance(x[key],str) and not isEnglish(x[key]):
        #     continue
        # 北京保留中文
        # if key == 'name' and not isEnglish(x['name'].iloc[0]):
        #     continue
        if not x[key].isna().iloc[0]:
            # print(x[key])
            sen = sen + key +" is " + x[key].iloc[0] + ","
    # print(sen)
    if not x['other_tags'].isna().iloc[0]:
        tags = x['other_tags'].iloc[0].split("\",\"")
        for tag in tags:
            try:
                tag = tag.replace("\"","")
                t = tag.split("=>")[0]
                k = tag.split("=>")[1]
                # 北京保留中文
                # if not isEnglish(k) or not isEnglish(t):
                    # print(k,t)
                    # continue
                if list(t)[-1] == "=":
                    continue
                if t in tag_ignore_list:
                    continue
                # if "name:" in t and t != "name:en":
                #     pass
                # pass 的作用是什么也不做 跳过，continue的作用是执行下次for循环
                # print(t,k)
                sen = sen + t +" is " + k + ","
            except:
                pass

    # sen = sen + "located in " + x['NAME_2'].iloc[0]
    
    # return sen,b
    
    return sen

In [ ]:
# a="脿陇陇脿陇戮脿陇聹脿陇庐脿陇鹿脿陇戮脿陇脿陇陋脿楼聟脿陇虏脿楼聡脿陇脿陇鹿脿楼聣脿陇聼脿楼聡脿陇"
# # print(isEnglish(a))
# b="à¤­à¤¾à¤°à¤¤à¥à¤à¤¹à¤µà¤¾à¤®à¤¾à¤¨ à¤µà¤¿à¤­à¤¾à¤"
# print(isEnglish(b))
# print(b.encode('utf-8').decode('ISO-8859-1'))
# s = "啶ぞ啶班い啷€啶啶灌さ啶距ぎ啶距え 啶掂た啶ぞ啶"
# print(s.encode('ansi').decode('iscii'))

将poi对应的自然语言提示存储到文件中(poi_nec最后插入一列提示)

In [82]:
def store_poi(x):
    # print(x)
    poi = x[0]
    # print(poi)    
    poi_im = poi_df_nec.loc[poi_df_nec['id']==poi]
    sen = genSent(poi_im)
    return sen
    # print(sen)


    

In [83]:
from tqdm import tqdm
result =[]
for index, row in tqdm(poi_df_nec.iterrows(), total=poi_df_nec.shape[0], desc="Processing"):
    # if index == 130:
    #     print(store_poi(row))
    result.append(store_poi(row))
# result = poi_df_nec.apply(lambda x: store_poi(x), axis=1, result_type='expand')
# poi_df_nec['poi_prompt']= result
# poi_df_nec['poi_prompt'] = result

Processing: 100%|██████████| 96107/96107 [04:03<00:00, 394.05it/s]


In [84]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec3.csv")
poi_df_nec['poi_prompt'] = result
poi_df_nec.to_csv(poi_prompt_path)
poi_df_nec

,id,name,barrier,highway,address,place,man_made,other_tags,poi_prompt
0,1,北京市,NaN,NaN,NaN,city,NaN,"""admin_level""=>""2"",""alt_name""=>""北京"",""alt_name:cs""=>""Pej-ťing"",""alt_name:en""=>""Peking;Peiping Municipality;Pei-p'ing Shih;Peking Municipality;Peiping Municipal Administrative Area"",""alt_name:eo""=>""Bejĝino"",""alt_name:fr""=>""Beijing"",""alt_name:is""=>""Pe","Nearby,name is 北京市,place is city,admin_level is 2,alt_name is 北京,alt_name:en is Peking;Peiping Municipality;Pei-p'ing Shih;Peking Municipality;Peiping Municipal Administrative Area,alt_name:eo is Bejĝino,alt_name:fr is Beijing,alt_name:is is Pe,"
1,2,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward""","Nearby,highway is traffic_signals,traffic_signals:direction is forward,"
2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward""","Nearby,highway is traffic_signals,traffic_signals:direction is forward,"
3,4,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward""","Nearby,highway is traffic_signals,traffic_signals:direction is forward,"
4,5,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals:direction""=>""forward""","Nearby,highway is traffic_signals,traffic_signals:direction is forward,"
...,...,...,...,...,...,...,...,...,...
96102,96103,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal""","Nearby,highway is traffic_signals,traffic_signals is signal,"
96103,96104,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal""","Nearby,highway is traffic_signals,traffic_signals is signal,"
96104,96105,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal""","Nearby,highway is traffic_signals,traffic_signals is signal,"
96105,96106,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal""","Nearby,highway is traffic_signals,traffic_signals is signal,"


In [87]:
def find_best_poi(x):
    # print(x)
    img_key = x['img_key']
    # img_file = img_key+'.jpg'
    img_file =img_key
    try:
        img_desc = desc_pd.loc[desc_pd['key']==img_file]['prompt'].iloc[0]
    except:
        print("The description of picture " + img_file +" doesn't exist")
        return  None,None
    pois = x[1::2]
    pois_dis =x[2::2]
    # print(pois)
    similarities = []
    sentences = []
    for index,poi in enumerate(pois):
        # 设置距离阈值
        if pois_dis[index] is not None and pois_dis[index]<100:

            # # 根据id从poi_df_nec中找到poi_prompt
            # try:
            #     sen = poi_df_nec.loc[poi_df_nec['id']==poi]['poi_prompt'].iloc[0]
            # except Exception as e:
            #     print(e)
            #     print("The poi_prompt of poi " + poi +" doesn't exist")
            #     continue
            
            # poi_im = poi_df_nec.loc[poi_df_nec['id']==poi]
            # sen = genSent(poi_im)
            try:
            # 检查 DataFrame 是否为空
                if not poi_df_nec.loc[poi_df_nec['id']==poi].empty:
                    sen = poi_df_nec.loc[poi_df_nec['id']==poi]['poi_prompt'].iloc[0]
                # 在此处理 sen 的其他操作
                else:
                    print("No matching rows for poi: " + poi)
                    continue
            except Exception as e:
                print(e)
                print("An error occurred while processing poi " + poi)
                continue

        
            similarity = calc_similarity(img_desc,sen)
            sentences.append(sen)
            similarities.append(similarity)
        
    if len(similarities) == 0:
        best_poi=None
        return best_poi,0
    elif similarities and max(similarities)>0.5:
        best_poi = pois[similarities.index(max(similarities))]
        return best_poi,max(similarities)

In [88]:
result = []
for _, row in tqdm(pic_poi_df.iterrows(), total=pic_poi_df.shape[0], desc="Processing"):
    result.append(find_best_poi(row))

pic_poi_df[['best_poi', 'best_poi_score']] = result

Processing:   0%|          | 0/132878 [00:00<?, ?it/s]

Processing:   0%|          | 8/132878 [00:02<10:37:46,  3.47it/s]

single positional indexer is out-of-bounds


TypeError: can only concatenate str (not "float") to str

In [ ]:
# 上一块做了，这一块就不做了
#result = pic_poi_df.apply(lambda x: find_best_poi(x), axis=1,result_type='expand')

In [ ]:
#pic_poi_df[['best_poi','similarity']]= result

In [63]:
pic_poi_df

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,distance5,best_poi,best_poi_score
0,002X1xuI4XOHBir7-owySg,11362,79.865680,70509.0,82.517518,70507.0,90.497904,70508.0,96.740065,59698.0,113.766971,11362.0,0.697060
1,003EDEceaZbom1pL5BT71g,36664,70.387451,75413.0,235.091576,36665.0,324.879509,21346.0,376.064179,21348.0,378.348055,36664.0,0.609343
2,005mgSHE0DfmONwbkZrE5A,39296,79.438995,39354.0,116.160086,39295.0,117.426100,38720.0,161.605241,38719.0,163.052673,39296.0,0.853308
3,006xgYXlcHgZ8BfTttGuIA,58185,66.851118,11517.0,80.813306,58178.0,83.479790,54082.0,84.786197,11519.0,87.400682,11517.0,0.780802
4,007x7NqVxRpQWenELoWNiw,27457,81.035875,49286.0,155.152377,27564.0,166.277333,44452.0,167.483308,27498.0,180.348641,27457.0,0.841849
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132873,ZzZb1yoT-DYIk-Bvb1KI_Q,82356,24.981663,7072.0,37.628208,7073.0,39.040349,2692.0,39.958836,82355.0,45.165065,7072.0,0.774242
132874,Zzzl07EjELpMNuScHuWvOQ,85607,309.203806,85608.0,318.729493,85610.0,332.109423,85609.0,339.779948,NaN,NaN,NaN,0.000000
132875,ZZzwGOYsu3J8NwRqR7dvGg,20801,7.442443,21698.0,30.921744,19616.0,68.250814,15765.0,121.839202,22496.0,146.615834,19616.0,0.883109
132876,ZzZX6Ac03dml7i9NUiabGg,18739,5.880840,18734.0,8.979344,18707.0,16.072906,18717.0,17.658410,18713.0,23.614230,18739.0,0.757936


In [64]:
pic_poi_df.to_csv(best_poi_path)

In [65]:
df1 = pd.merge(pic_poi_df, poi_df_nec, left_on="best_poi",right_on = 'id',how="left")

In [66]:
df1

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,...,best_poi_score,id,name,barrier,highway,address,place,man_made,other_tags,poi_prompt
0,002X1xuI4XOHBir7-owySg,11362,79.865680,70509.0,82.517518,70507.0,90.497904,70508.0,96.740065,59698.0,...,0.697060,11362.0,月坛体育场,NaN,NaN,NaN,NaN,NaN,"""bus""=>""yes"",""name:zh""=>""月坛体育场"",""public_transport""=>""stop_position""","Nearby,bus is yes,public_transport is stop_position,"
1,003EDEceaZbom1pL5BT71g,36664,70.387451,75413.0,235.091576,36665.0,324.879509,21346.0,376.064179,21348.0,...,0.609343,36664.0,东门,gate,NaN,NaN,NaN,NaN,"""addr:city""=>""北京"",""addr:housenumber""=>""1"",""addr:postcode""=>""102628"",""addr:street""=>""安顺北路"",""bicycle""=>""yes"",""foot""=>""yes"",""horse""=>""no"",""motor_vehicle""=>""yes""","Nearby,barrier is gate,bicycle is yes,foot is yes,horse is no,motor_vehicle is yes,"
2,005mgSHE0DfmONwbkZrE5A,39296,79.438995,39354.0,116.160086,39295.0,117.426100,38720.0,161.605241,38719.0,...,0.853308,39296.0,NaN,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""restaurant""","Nearby,amenity is restaurant,"
3,006xgYXlcHgZ8BfTttGuIA,58185,66.851118,11517.0,80.813306,58178.0,83.479790,54082.0,84.786197,11519.0,...,0.780802,11517.0,新南21号旁,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""toilets"",""opening_hours""=>""Mo-Su 05:00-24:00"",""operator""=>""北京市西城区环境卫生服务中心""","Nearby,amenity is toilets,"
4,007x7NqVxRpQWenELoWNiw,27457,81.035875,49286.0,155.152377,27564.0,166.277333,44452.0,167.483308,27498.0,...,0.841849,27457.0,NaN,NaN,NaN,NaN,NaN,NaN,"""power""=>""tower""","Nearby,power is tower,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132873,ZzZb1yoT-DYIk-Bvb1KI_Q,82356,24.981663,7072.0,37.628208,7073.0,39.040349,2692.0,39.958836,82355.0,...,0.774242,7072.0,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
132874,Zzzl07EjELpMNuScHuWvOQ,85607,309.203806,85608.0,318.729493,85610.0,332.109423,85609.0,339.779948,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132875,ZZzwGOYsu3J8NwRqR7dvGg,20801,7.442443,21698.0,30.921744,19616.0,68.250814,15765.0,121.839202,22496.0,...,0.883109,19616.0,眉州东坡酒楼,NaN,NaN,NaN,NaN,NaN,"""amenity""=>""restaurant""","Nearby,amenity is restaurant,"
132876,ZzZX6Ac03dml7i9NUiabGg,18739,5.880840,18734.0,8.979344,18707.0,16.072906,18717.0,17.658410,18713.0,...,0.757936,18739.0,NaN,NaN,NaN,NaN,NaN,NaN,"""natural""=>""tree""","Nearby,natural is tree,"


In [67]:
df1.to_csv(best_poi_path)

In [68]:

poi_df_nec.columns

Index(['id', 'name', 'barrier', 'highway', 'address', 'place', 'man_made',
       'other_tags', 'poi_prompt'],
      dtype='object')

In [69]:
# desc_pd: file,prompt insert poi_prompt
# df1 : image_key, bestpoi
# poi_df_nec: FID ,poi_prompt
def connect(x):
    image_file_name = x[0]
    image_name =image_file_name.split('.')[0]
    best_poi_row = df1.loc[df1['img_key']==image_name]['best_poi']
    best_poi = None
    if best_poi_row.notna().any():
        best_poi=int(best_poi_row.iloc[0])
        poi_prompt = poi_df_nec.loc[poi_df_nec['id']==best_poi]['poi_prompt'].iloc[0]
        return poi_prompt
        # print(poi_prompt)
    # print(best_poi)
    # print("\n")
    

    # print(image_name)

In [70]:
result2 = desc_pd.apply(lambda x: connect(x), axis=1,result_type='expand')
result2

0         Nearby,name:en is Buddha Incense Gallery,tourism is attraction,
1         Nearby,name:en is Longevity Hill,natural is peak,              
2         Nearby,name:en is Buddha Incense Gallery,tourism is attraction,
3         Nearby,name:en is Buddha Incense Gallery,tourism is attraction,
4         Nearby,name:en is Buddha Incense Gallery,tourism is attraction,
                                       ...                               
157997    None                                                           
157998    None                                                           
157999    Nearby,highway is motorway_junction,                           
158000    None                                                           
158001    None                                                           
Length: 158002, dtype: object

In [71]:
desc_pd['poi_prompt'] = result2

In [72]:
desc_pd

,key,prompt,poi_prompt
0,10pghDAJsxu1sdFN-xwKNg,"there is a cat sitting on the steps in front of a building, a detailed matte painting inspired by Shin Saimdang, tumblr, cloisonnism, korean traditional palace, beautiful ancient ruins behind, in forbidden city","Nearby,name:en is Buddha Incense Gallery,tourism is attraction,"
1,gvh8c4gE4__oDvbB46QhYw,"araffe with a lot of people walking around it, a photo inspired by Gong Xian, tumblr, cloisonnism, ancien chinese tower, taoist temples and monks, chinese architecture","Nearby,name:en is Longevity Hill,natural is peak,"
2,gGhf1zLc7LNTG_xNCU-P4w,"arafed roof of a building with a view of a river and trees, a detailed matte painting inspired by Gong Xian, tumblr, cloisonnism, imperial city in the distance, chinese palace, ancien chinese tower","Nearby,name:en is Buddha Incense Gallery,tourism is attraction,"
3,kYJWkBuWPPrjkT7bhBcIvQ,"there is a view of a building with a green roof and a yellow railing, a matte painting by Emperor Huizong of Song, cg society, cloisonnism, heise jinyao, taoist temples and monks, pagodas on hills","Nearby,name:en is Buddha Incense Gallery,tourism is attraction,"
4,zEp8lLNHY4m4kNjdI0Liow,"people walking up a steep staircase with a view of a lake, a matte painting inspired by Chen Daofu, dribbble, cloisonnism, 400 steps, taoist temples and monks, pagodas on hills","Nearby,name:en is Buddha Incense Gallery,tourism is attraction,"
...,...,...,...
157997,ClJqUg9jNDQh9ORQ6ufn7A,"cars are driving down a snowy street in a city, a matte painting by Muqi, polycount, dau-al-set, baotou china, artificial fog, sky is not visible",None
157998,XbHTwE8c6KY-NXyAc-Ur2Q,"snowy street with cars driving on it in the middle of the day, a matte painting by Weiwei, tumblr, constructivism, neo norilsk, baotou china, soviet suburbs",None
157999,WfvUbbt2GKVF29yV0rcupg,"arafed view of a highway with a foggy sky and street lights, a matte painting by Ye Xin, tumblr, tonalism, foggy photo 8 k, gloomy and foggy atmosphere, gloomy foggy atmosphere","Nearby,highway is motorway_junction,"
158000,xIBlWSCw8BVvPzlv8q1cJw,"arafed view of a car driving down a snowy road, a matte painting inspired by Zhang Kechun, tumblr, tonalism, in winter, heise jinyao, winter photograph",None


In [73]:
desc_pd.to_csv(desc_poi_path)